In [1]:
import pandas as pd
import networkx as nx
import itertools
import collections

### Load the dataset

In [2]:
animes_df = pd.read_csv("Anime.csv")

### Producers network

In [6]:
# split the producers
animes_producers_df = animes_df.assign(Producers=animes_df["Producers"].str.split(", ")).explode("Producers")
# remove the unknown producer rows
animes_producers_df = animes_producers_df[animes_producers_df["Producers"]!="Unknown"]
# find the producer cooperation pairs
all_pairs = []
animes_producers_only_df = animes_producers_df.groupby("ID")["Producers"].agg([lambda x: set(x)]).reset_index().rename(columns={"<lambda>":"Producer Set"})
for i in list(animes_producers_only_df["Producer Set"]):
    all_pairs.extend([frozenset(j) for j in itertools.combinations(i, 2)])
all_pairs_weight_dict = dict(collections.Counter(all_pairs))
# create the producer dataframe used for network
producers_weighted_df = pd.DataFrame.from_dict(all_pairs_weight_dict, orient="index").reset_index().rename(columns={"index":"tuple",0:"weight"})
producers_weighted_df[["Src", "Dest"]] = pd.DataFrame(producers_weighted_df["tuple"].tolist(), index=producers_weighted_df.index)
# create the network
producers_network = nx.from_pandas_edgelist(producers_weighted_df, "Src", "Dest", ["weight"])
print(nx.info(producers_network))
nx.write_gml(producers_network, "networks/producers_network.gml")

Name: 
Type: Graph
Number of nodes: 1224
Number of edges: 13434
Average degree:  21.9510
